# 5. Reducción dataset

El objetivo de este notebook es procesar el dataset que se ha utilizado para entrenar le modelo preliminar y tratar de reducir la dimensionalidad de este sin perder precisión. De esta forma, se identificarán que sensores no son necesarios en la máquina para identificar posibles reducciones de costes.

## 5.1. Dataset y modelo inicial

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

def evaluar_precision(modelo, X, y, cv=5):
    """
    Entrena un modelo usando cross-validation y retorna la precisión media.
    :param modelo: instancia de un estimador de scikit-learn (por ej. RandomForestClassifier).
    :param X: features (DataFrame o NumPy array).
    :param y: variable objetivo (Series o array).
    :param cv: número de folds para la validación cruzada.
    :return: None (imprime la precisión media y desviación estándar).
    """
    scores = cross_val_score(modelo, X, y, cv=cv, scoring='accuracy')
    print(f"Precisión media (CV={cv}): {scores.mean():.4f} ± {scores.std():.4f}")


def evaluar_metricas(modelo, X, y, cv=5):
    """
    Entrena un modelo usando cross-validation y reporta Accuracy, Recall y F1.
    """
    scoring = {
        'accuracy': 'accuracy',
        'recall': 'recall_macro',   # o 'recall_weighted' si lo prefieres
        'f1': 'f1_macro'            # o 'f1_weighted'
    }
    resultados = cross_validate(modelo, X, y, cv=cv, scoring=scoring)
    print(f"Accuracy (CV={cv}): {resultados['test_accuracy'].mean():.4f} ± {resultados['test_accuracy'].std():.4f}")
    print(f"Recall   (CV={cv}): {resultados['test_recall'].mean():.4f} ± {resultados['test_recall'].std():.4f}")
    print(f"F1       (CV={cv}): {resultados['test_f1'].mean():.4f} ± {resultados['test_f1'].std():.4f}")


In [2]:
df = pd.read_csv("ultimate.csv")
print("El dataset tiene incialmente un tamaño de:")

El dataset tiene incialmente un tamaño de:


En primer lugar, se eliminan las columnas "Tipo", "Hz" y "medida", ya que no son columnas que se utilizan para el entrenamiento.
Antes de comenzar la reducción de dimensionalidad del dataset de entrenamiento, el modelo de Random Forest tiene una precisión de:

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

X = df.drop(columns=['Tipo', 'Hz', 'medida'])  # Ejemplo, quitamos las que no van al modelo
y = df['Tipo']

modelo = RandomForestClassifier(n_estimators=100, random_state=42)

print("=== Métricas sin reducción de dimensionalidad ===")
evaluar_precision(modelo, X, y)  # o evaluar_metricas(modelo, X, y)


=== Métricas sin reducción de dimensionalidad ===
Precisión media (CV=5): 0.8880 ± 0.0471


## 5.2. Filtrado inicial
Reducción de dimensionalidad no supervisada
### 5.2.1. Eliminar variables de baja varianza
Algunas variables pueden presentar muy poca variación (casi todos los valores iguales), lo que las hace poco relevantes para la predicción.

In [4]:
from sklearn.feature_selection import VarianceThreshold

selector_var = VarianceThreshold(threshold=0.01)
selector_var.fit(X)
cols_survived = X.columns[selector_var.get_support()]
df_reduced_1 = X[cols_survived]

print("\n=== Métricas tras eliminar varianza muy baja ===")
evaluar_precision(modelo, df_reduced_1, y)

print("\nTras reducción nº1, l dataset tiene un tamaño de:")
df_reduced_1.shape


=== Métricas tras eliminar varianza muy baja ===
Precisión media (CV=5): 0.8833 ± 0.0498

Tras reducción nº1, l dataset tiene un tamaño de:


(84000, 61)

### 5.2.2. Análisis de correlación
Si hay variables altamente correlacionadas entre sí (multicolinealidad), puede bastar con quedarte con una de ellas.

In [5]:
import numpy as np

# Matriz de correlaciones
corr_matrix = df_reduced_1.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Umbral de 0.9 (ejemplo)
to_drop = [c for c in upper_tri.columns if any(upper_tri[c] > 0.9)]
df_reduced_2 = df_reduced_1.drop(columns=to_drop)

print("\n=== Métricas tras eliminar correlaciones altas ===")
evaluar_precision(modelo, df_reduced_2, y)

print("\nTras reducción nº2, el dataset tiene un tamaño de:")
print(df_reduced_2.shape)


=== Métricas tras eliminar correlaciones altas ===
Precisión media (CV=5): 0.8778 ± 0.0467

Tras reducción nº2, el dataset tiene un tamaño de:
(84000, 32)


## 5.3. Métodos supervisados de selección de características
Estos métodos consideran la relación entre cada variable y la variable objetivo ("Tipo") para descartar características irrelevantes o redundantes.

### 5.3.1. Feature importance en modelos basados en árboles
* Entrenar un Random Forest o un XGBoost preliminarmente y obtener la importancia de cada variable.
* Con la importancia, filtrar aquellaspor debajo un cierto umbral o seleccionar las "top K" variables más relevantes.

In [6]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder
# from xgboost import XGBClassifier

# le = LabelEncoder()
# y_train_encoded = le.fit_transform(y_train)
# y_test_encoded = le.transform(y_test)   # para test, usar "transform", NO "fit_transform"

# model = XGBClassifier()
# model.fit(X_train, y_train_encoded)

# y_pred_encoded = model.predict(X_test)

# y_pred = le.inverse_transform(y_pred_encoded) # Descodificar para obtener los nombres originales

# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# print("\nAccuracy:", accuracy_score(y_test, y_pred))
# print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))
# print("Reporte de Clasificación:\n", classification_report(y_test, y_pred))


### 5.3.2. Selección por modelos lineales con regularización L1 (Lasso)
* Aplicar un modelo lineal (por ejemplo, Logistic Regression con penalización L1) que fuerce a ciertos coeficientes a ser cero y elimina variables poco relevantes.

In [7]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectFromModel

# estimator = LogisticRegression(penalty='l1', solver='saga', max_iter=84000)
# selector = SelectFromModel(estimator=estimator, threshold='mean')
# selector.fit(X_train, y_train)

# X_train_reduced = selector.transform(X_train)
# X_test_reduced  = selector.transform(X_test)

In [8]:
# print("Tamaño después de procesar")
# print(X_train_reduced.shape)
# print(X_test_reduced.shape)

### 5.3.3. Wrapper methods: Recursive Feature Elimination (RFE)
* Consiste en entrenar un modelo y eliminar iterativamente las características 

In [9]:
# from sklearn.feature_selection import RFE
# from sklearn.ensemble import RandomForestClassifier

# estimator = RandomForestClassifier(n_estimators=100)
# rfe = RFE (estimator, n_features_to_select=20)
# rfe.fit(X_train, y_train)

# X_train_reduced = rfe.transform(X_train)
# X_test_reduced = rfe.transform(X_test)

## 5.4. Métodos de proyección
En estos casos, las variables originales se combinan (mediante proyecciones lineales o no lineales) para reducir la dimensión. Algunas técnicas son:

### 5.4.1. Principal component Analysis (PCA)
* Método no supervisado que busca maximizar la varianza en los primeros componentes principales.
* Útil cuando hay mucho colinealidad y el dataset es muy grande.
* El inconveniente es que las nuevas variables (componentes principales) pueden perder interpretabilidad directa.

In [10]:
# from sklearn.decomposition import PCA

# # Por ejemplo, 20 componentes
# # 10 componentes da 0.60 de precisión
# # 20 componentes da 0.80 de precisión
# # 25 componentes da 0.82 de precisión
# # 30 componentes da 0.82

# pca = PCA(n_components=30, random_state=42)
# df_reduced_3 = pca.fit_transform(df_reduced_2)

# print("\n=== Métricas tras PCA a 30 componentes ===")
# evaluar_precision(modelo, df_reduced_3, y)
# # evaluar_metricas(modelo, df_reduced_3,y)

# print("\nTras reducción nº3, el dataset tiene un tamaño de:")
# df_reduced_3.shape

print("Con PCA de 30 componentes, la precisión baja a 0,82. Por lo que no merece la pena aplicar esta técnica.")

Con PCA de 30 componentes, la precisión baja a 0,82. Por lo que no merece la pena aplicar esta técnica.


### 5.4.2. Linear Discriminant Analysis (LDA)
* Similar al PCA pero supervisado.
* Para problemas multiclass, LDA puede reducir la dimensionalidad hasta n_clases -1 componentes relevantes.
* Puede ser muy efectivo si las clases están bien separadas linealmente.

In [11]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# df_reduced_2

# n_clases = y.nunique()
# n_comp_max = min(df_reduced_2.shape[1], n_clases - 1)
# print("Número de clases:", n_clases)
# print(f"LDA puede tener hasta {n_comp_max} componentes.")

# lda = LDA(n_components=n_comp_max)
# df_reduced_4 = lda.fit_transform(df_reduced_2, y)

# print(f"\n=== Métricas tras LDA con {n_comp_max} componentes ===")
# evaluar_precision(modelo, df_reduced_4, y)

# print("\nTras reducción con LDA, el dataset tiene un tamaño de:")
# print(df_reduced_4.shape)

print("Con LDA de 9 componentes, la precisión baja a 0.68. Por lo que no merece la pena.")

Con LDA de 9 componentes, la precisión baja a 0.68. Por lo que no merece la pena.


## 5.5. Comprobar precisión después de reducción de dimensionalidad
Como conclusión, el flujo de técnicas que se han utilizado para la reducción de dimensionalidad es el siguiente:
1. Un filtrado inicial (varianza y correlación),
2. Método supervisado (por ejemplo, importancia de variables en Random Forest / XGBoost, o un modelo lineal con L1)
3. PCA o LDA

In [12]:
# evaluar_precision(modelo, df_reduced_2, y)
# evaluar_metricas(modelo, df_reduced_2, y)

print("\nEl dataset final tiene un tamaño de:")
df_reduced_2.shape



El dataset final tiene un tamaño de:


(84000, 32)

In [13]:
# Convertir las columnas en sets
set_inicial = set(df.columns)
set_reducido = set(df_reduced_2.columns)

# Calcular y ordenar alfabéticamente las columnas eliminadas
columnas_eliminadas = sorted(list(set_inicial - set_reducido))

# Imprimir cada columna en una línea
print("Columnas eliminadas (ordenadas alfabéticamente):")
for columna in columnas_eliminadas:
    print(columna)



Columnas eliminadas (ordenadas alfabéticamente):
Hz
S1_max
S1_var
S2_IQR
S2_max
S2_min
S2_var
S3_IQR
S3_max
S3_median
S3_min
S4_IQR
S4_max
S4_min
S4_var
S5_IQR
S5_median
S5_var
S6_IQR
S6_max
S6_median
S6_min
S6_var
S7_IQR
S7_max
S7_median
S7_min
S7_var
S8_IQR
S8_mean
S8_median
S8_min
S8_var
Tipo
medida


No hay ningún sensor para el cual se han eliminado todos los estadísticos provenientes que parten

In [14]:
# Añadir columnas de vuelta a df_reduced_2
df_reduced_2["Hz"] = df["Hz"]
df_reduced_2["medida"] = df["medida"]
df_reduced_2["Tipo"] = df["Tipo"]

# Comprobamos el nuevo tamaño
print("Ahora df_reduced_2 tiene un tamaño de:")
print(df_reduced_2.shape)


df_reduced_2.to_csv("df_reduced.csv", index=False)

Ahora df_reduced_2 tiene un tamaño de:
(84000, 35)
